In [ ]:
%pip install python-dotenv
%pip install -U langchain langchain-google-genai langchain-tavily python-dotenv aiosqlite
%pip install --upgrade langgraph
%pip install langgraph-checkpoint-sqlite

In [21]:
import os
from dotenv import load_dotenv
import operator
from typing import Annotated, List, Any, Dict
from dataclasses import dataclass, field
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage,AIMessage, BaseMessage, AnyMessage
from datetime import date
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch
import uuid

from langgraph.checkpoint.sqlite import SqliteSaver

from typing_extensions import TypedDict
import sqlite3

load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [22]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

conn = sqlite3.connect("checkpoints.db", check_same_thread=False)
memory = SqliteSaver(conn)


In [23]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_gemini)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        ) 
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        
    def call_gemini(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages          
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Voltando para o modelo!")
        return {'messages': results}
    


In [29]:
current_date = date.today().strftime("%d/%m/%Y")
tool = TavilySearch(max_results=3)  # usa TAVILY_API_KEY do ambiente
prompt = f"""Você é um assistente de pesquisa inteligente e altamente atualizado. \
Sua principal prioridade é encontrar as informações mais RECENTES e em TEMPO REAL sempre que possível. \
A data atual é {current_date}. \
Ao buscar sobre o tempo ou eventos que se referem a "hoje" ou "agora", \
você DEVE **incluir a data atual `{current_date}` na sua consulta para a ferramenta de busca**. \
Por exemplo, se a pergunta é "tempo em cidade x hoje", a consulta para a ferramenta deve ser "tempo em cidade x {current_date}". \
Ignore ou descarte informações que claramente se refiram a datas passadas ou futuras ao responder perguntas sobre "hoje". \
Use o mecanismo de busca para procurar informações, sempre buscando o "hoje" ou o "agora" quando o contexto indicar. \
Você tem permissão para fazer múltiplas chamadas (seja em conjunto ou em sequência). \
Procure informações apenas quando tiver certeza do que você quer. \
Se precisar pesquisar alguma informação antes de fazer uma pergunta de acompanhamento, você tem permissão para fazer isso!
"""
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
abot = Agent(model, [tool], system = prompt, checkpointer = memory)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [25]:
dynamic_thread_id = str(uuid.uuid4())

print(f"Meu novo Thread ID dinâmico é: {dynamic_thread_id}")

Meu novo Thread ID dinâmico é: 459a720f-95db-4c04-8a13-926c52be8e38


In [30]:
def extract_text(msg):
    if isinstance(msg.content, list):
        return "".join(
            part.get("text", "")
            for part in msg.content
            if part.get("type") == "text"
        )
    return msg.content

session_id = str(uuid.uuid4())
print(f'DEBUG: Iniciando nova conversa com ID: {session_id}\n')

user_mesage = "Como está o tempo em São Paulo hoje (21/08/2025)?"
messages = [HumanMessage(content=user_mesage)]
thread_config = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n--- Etapa 1: Agente processa a entrada e decide a ação ---")
print(f"Você: {user_mesage}")

for event in abot.graph.stream({"messages": messages}, thread_config): 

    for k, v in event.items(): 
        if k == "llm":
            last_message = v["messages"][-1]
            print(extract_text(last_message))
            if isinstance (last_message, AIMessage) and last_message.tool_calls: 
                print(f'\nAgente (decisão): {extract_text(last_message)}') 
                print("\n --- AGENTE PAUSADO: Intervenção Humana Necessaria ---") 
            else: 
                print(f'\nAgente (resposta direta/sem tool_cals): {extract_text(last_message)}') 
                print("\n --- AGENTE PAUSADO: (resposta direta, sem ação pendente) ---") 
current_state = abot.graph.get_state(thread_config)

last_state_message = current_state.values['messages'][-1]

if current_state and current_state.next == ('action',) and isinstance (last_state_message, AIMessage) and last_state_message.tool_calls:
    tool_calls_pending = last_state_message.tool_calls
    if tool_calls_pending:
        print('\nO agente decidiu executar a(s) seguinte(s) ação(ões) de ferramenta:')
        for tc in tool_calls_pending:
            print(f'- Ferramenta: {tc["name"]}, Argumentos: {tc["args"]}')

        user_input = input('\nVocê deseja que o agente execute esta(s) ação(ões)? (sim/não)').lower()

        if user_input == 'sim':
            print('\n--- Etapa2: Remotando a execução (Agente executaré a ação) ---')
            for event in abot.graph.stream(None, thread_config):
                for k, v in event.items():
                    if k == "llm":
                        last_message = v["messages"][-1]
                        print(extract_text(last_message))
                    elif k == 'llm':
                        final_response_message = v.get('messages', [])[-1].content
                        print(f'\n Agente (resposta final): {extract_text(v["messages"][-1])}')
                    elif k == END:
                        print(f'DEBUG: Grafo terminou a execução.')
            print("--- FIM DA INTERAÇÃO ---")
        else:
            print("Execução cancelada pelo usuário")
            print("--- FIM DA INTERAÇÃO ---")
    else:
        print("\nO Agente não decidiu nenhuma ação de ferramenta apesar da pausa. Interação encerrada.")

else:
    print("\nO Agente respondeu diretamente ou não pausou nenhuma ação. Não há ações pendentes para aprovar.")
    if current_state:
        final_response_message = current_state.values['messages'][-1].content
        print(f'Agente (resposta direta): {final_response_message.content[0]["text"]}')
    print("--- FIM DA INTERAÇÃO ---")

DEBUG: Iniciando nova conversa com ID: 90d17b54-2edc-42fc-8d80-ef32393b132d


--- Etapa 1: Agente processa a entrada e decide a ação ---
Você: Como está o tempo em São Paulo hoje (21/08/2025)?


Agente (decisão): 

 --- AGENTE PAUSADO: Intervenção Humana Necessaria ---

O agente decidiu executar a(s) seguinte(s) ação(ões) de ferramenta:
- Ferramenta: tavily_search, Argumentos: {'query': 'tempo em São Paulo 21/08/2025', 'search_depth': 'basic'}

--- Etapa2: Remotando a execução (Agente executaré a ação) ---
Voltando para o modelo!
Para 21/08/2025 em São Paulo, as previsões do tempo são um pouco variadas:

*   Algumas fontes indicam céu nublado com pancadas rápidas de chuva, com temperaturas entre 16°C e 32°C.
*   Outras preveem um dia ensolarado, com temperaturas mínimas entre 19°C e 20.1°C e máximas entre 32.1°C e 33°C, sem previsão de chuva e com umidade do ar em torno de 23%.

Devido às informações conflitantes, não é possível dar uma previsão exata.
--- FIM DA INTERAÇÃO ---
